In [107]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns


## CLEANING DATASET

In [108]:
df = pd.read_csv('scraped_data_alonhadat_onlyfront_1.csv',encoding='utf-8')
df

,price,area,road_width,floors,bedroom,loc,garage
0,": 3,8 tỷ",50 m\n2,7m,2 lầu,2 phòng ngủ,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
1,": 9,7 tỷ",100 m\n2,6m,2 lầu,3 phòng ngủ,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",chỗ để xe
2,: 24 tỷ,128 m\n2,"7,5m",3 lầu,3 phòng ngủ,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",chỗ để xe
3,": 25,5 tỷ",165 m\n2,"7,5m",1 lầu,1 phòng ngủ,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0
4,: 24 tỷ,320 m\n2,30m,5 lầu,14 phòng ngủ,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
...,...,...,...,...,...,...,...
8628,": 4,6 tỷ",42 m\n2,"5,5m",4 lầu,4 phòng ngủ,"Đường Hoàng Kế Viêm, Phường Mỹ An, Quận Ngũ Hà...",chỗ để xe
8629,": 8,499 tỷ",70 m\n2,"5,5m",2 lầu,3 phòng ngủ,"Đường An Thượng 17, Phường Mỹ An, Quận Ngũ Hàn...",chỗ để xe
8630,: 10 tỷ,100 m\n2,5m,5 lầu,10 phòng ngủ,"Đường Chế Lan Viên, Phường Mỹ An, Quận Ngũ Hàn...",chỗ để xe
8631,": 9,3 tỷ",90 m\n2,18m,3 lầu,4 phòng ngủ,"Đường Hồ Xuân Hương, Phường Khuê Mỹ, Quận Ngũ ...",chỗ để xe


In [109]:
def clean_price(price, area):
    if isinstance(price, str):
        price = price.replace(':', '').replace(',', '.').strip()

        if 'ngàn' in price:
            return None
        
        if 'triệu/m2' in price and isinstance(area, str):
            try:
                area_value = clean_area(area)
                price_value = float(price.replace('triệu/m2', '').strip())
                return price_value * area_value / 1000  # convert 
            except:
                return None

        if 'tỷ' in price:
            try:
                return float(price.replace('tỷ', '').strip())
            except:
                return None
        
        if '/ m2' in price:
            try:
                price_value = price.split('/')[0].strip()
                return float(price_value)
            except:
                return None

    return None
def clean_area(area):
    if isinstance(area, str):
        area = area.replace('\n', '').replace('m', '').replace('2', '').strip()  # remove any "m" and "²" symbols
        try:
            return float(area)
        except:
            return None
    return None

def clean_numeric_column(value):
    if isinstance(value, str):
        try:
            return float(''.join([c for c in value if c.isdigit() or c == '.']))
        except:
            return None
    return None

def clean_garage(garage):
    if isinstance(garage, str) and "0" not in garage:
        return 1
    return 0

In [110]:
df['price'] = df.apply(lambda row: clean_price(row['price'], row['area']), axis=1)
df['area'] = df['area'].apply(clean_area)
df['garage'] = df['garage'].apply(clean_garage)
df['road_width'] = df['road_width'].apply(clean_numeric_column)
df['bedroom'] = df['bedroom'].apply(clean_numeric_column)
df['floors'] = df['floors'].apply(clean_numeric_column)


In [111]:
df['price']=df['price'].fillna(df['price'].mean())
df['area']=df['area'].fillna(df['area'].mean())
# df['road_width']=df['road_width'].fillna(df['road_width'].mean())
df['floors']=df['floors'].fillna(int(df['floors'].mean()))
df['bedroom']=df['bedroom'].fillna(int(df['bedroom'].mean()))

In [125]:
df.head(10)

,price,area,road_width,floors,bedroom,loc,garage
0,3.80,50.0,7.0,2.0,2.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
1,9.70,100.0,6.0,2.0,3.0,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",1
2,24.00,18.0,75.0,3.0,3.0,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",1
3,25.50,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0
4,24.00,30.0,30.0,5.0,14.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
5,28.00,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",1
6,6.05,100.0,75.0,2.0,3.0,"Đường Giang Châu 1, Phường Khuê Mỹ, Quận Ngũ H...",0
7,24.00,147.0,15.0,3.0,3.0,"Đường Đống Đa, Phường Thuận Phước, Quận Hải Châu,",1
8,5.60,90.0,75.0,3.0,4.0,"Đường Lê Đại Hành, Phường Khuê Trung, Quận Cẩm...",0
9,20.50,170.0,15.0,1.0,1.0,"Đường Hồ Nghinh, Phường Phước Mỹ, Quận Sơn Trà,",1


In [117]:
df = df.dropna()

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8621 entries, 0 to 8632
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       8621 non-null   float64
 1   area        8621 non-null   float64
 2   road_width  8621 non-null   float64
 3   floors      8621 non-null   float64
 4   bedroom     8621 non-null   float64
 5   loc         8621 non-null   object 
 6   garage      8621 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 538.8+ KB


In [119]:
df.to_csv('First_clean.csv',encoding='utf-8',index=False)

In [140]:
df_test = df[0:100]

## IMPORTING DATA INTO DATABASE

In [142]:
import pymysql


timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="defaultdb",
  host="mysql-for-ady201m-project-ady201m-nhom5.e.aivencloud.com",
  password="AVNS_Z2RBipwQfoNZD7N0O11",
  read_timeout=timeout,
  port=22064,
  user="avnadmin",
  write_timeout=timeout,
)
  

cursor = connection.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Housing_Data_1 (
        id INT PRIMARY KEY AUTO_INCREMENT,
        price FLOAT,
        area FLOAT,
        road_width FLOAT,
        floors FLOAT,
        bedroom INT,
        garage INT,
        loc TEXT
    )
""")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Housing_Data_1
        (price, area, road_width, floors, bedroom, garage, loc)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['price'], 
        row['area'], 
        row['road_width'], 
        row['floors'], 
        row['bedroom'], 
        row['garage'], 
        row['loc']
    ))

    # 
cursor.execute("SELECT * FROM Housing_Data_1")
results = cursor.fetchall()
for result in results:
    print(result)



[{'id': 8622, 'price': 3.8, 'area': 50.0, 'road_width': 7.0, 'floors': 2.0, 'bedroom': 2, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}, {'id': 8623, 'price': 9.7, 'area': 100.0, 'road_width': 6.0, 'floors': 2.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Trần Tống, Phường Thạc Gián, Quận Thanh Khê,'}, {'id': 8624, 'price': 24.0, 'area': 18.0, 'road_width': 75.0, 'floors': 3.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận Sơn Trà,'}, {'id': 8625, 'price': 25.5, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 0, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hành Sơn,'}, {'id': 8626, 'price': 24.0, 'area': 30.0, 'road_width': 30.0, 'floors': 5.0, 'bedroom': 14, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}, {'id': 8627, 'price': 28.0, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 1, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận

In [143]:
connection.commit()

In [144]:

for result in results:
    print(result)

{'id': 8622, 'price': 3.8, 'area': 50.0, 'road_width': 7.0, 'floors': 2.0, 'bedroom': 2, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}
{'id': 8623, 'price': 9.7, 'area': 100.0, 'road_width': 6.0, 'floors': 2.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Trần Tống, Phường Thạc Gián, Quận Thanh Khê,'}
{'id': 8624, 'price': 24.0, 'area': 18.0, 'road_width': 75.0, 'floors': 3.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận Sơn Trà,'}
{'id': 8625, 'price': 25.5, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 0, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hành Sơn,'}
{'id': 8626, 'price': 24.0, 'area': 30.0, 'road_width': 30.0, 'floors': 5.0, 'bedroom': 14, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}
{'id': 8627, 'price': 28.0, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 1, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ H